# Neural Networks

In this part of the exercise, you will implement a neural network to recognize handwritten digits using the MNIST dataset. The neural network will be able to represent complex models that form non-linear hypotheses.

## Dataset

You are given a data set in [`data_nn.npy`](https://goo.gl/GhGLAQ) that contains 5000 training examples of handwritten digits (This is a subset of the [MNIST](http://yann.lecun.com/exdb/mnist/) handwritten digit dataset. The `.npy` format means that that the data has been saved in a native Numpy matrix format, instead of a text (ASCII) format like a csv-file. These matrices can be read directly into your program by using the load command. After loading, matrices of the correct dimensions and values will appear in your program's memory.

There are 5000 training examples in `data_nn.npy`, where each training example is a 20 pixel by 20 pixel grayscale image of the digit. Each pixel is represented by a floating point number indicating the grayscale intensity at that location. The 20 by 20 grid of pixels is "unrolled" into a 400-dimensional vector. Each of these training examples becomes a single row in our data matrix `X`. This gives us a 5000 by 400 matrix `X` where every row is a training example for a handwritten digit image.

The second part of the training set is a 5000-dimensional vector `y` that contains labels for the training set.

In [ ]:
# Load dataset into matrices X and y
import numpy as np

def load_dataset(filedata):
    """ Load the content from data_nn.npy """
    data = np.load(filedata).item()
    X = data['X']
    y = data['y']
    return X, y

X, y = load_dataset('data/data_nn.npy')
X = np.matrix(X)
y = np.matrix(y)
print(X.shape)
print(y.shape)

## Preprocessing labels

As classification predicts scores for each class, first we have to convert the value of each class into a one-hot-encoding vector containing the value of the class. Thus, for a class `0` in our dataset, we represent it as one hot vector:

```
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
```

Class `1` in our dataset is represented as:


```
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
```

and so on.

In [ ]:
# Convert labels into one hot encoding
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False) # sparse=False return an array
y_onehot = encoder.fit_transform(y)
y_onehot = np.matrix(y_onehot)
y_onehot

## Visualizing the data

First thing you should do, is visualize the data loaded into `X`. To do so, select randomly 100 rows from `X` and map each row to a 20 pixel by 20 pixel grayscale image and display the images together. The resulting image should be like:

<img src='images/mnist.png' width="40%"/>

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

def display_image(X):
    """ From matrix X, display 100 random images into a single figure"""
    # YOUR CODE HERE
    pass

display_image(X)

## Model representation

Our neural network is shown in the image below. It has 3 layers - an input layer, a hidden layer and an output layer. Recall that our inputs are pixel values of digit images. Since the images are of size 20×20, this gives us 400 input layer units (excluding the extra bias unit which always outputs +1). 

You have been provided with a set of network parameters ($\theta^{(1)}$ , $\theta^{(2)}$) already trained. These are stored in [`weights_nn.npy`](https://goo.gl/grDJAS) and will be loaded into `theta1` and `theta2`. The parameters have dimensions that are sized for a neural network with 25 units in the second layer and 10 output units (corresponding to the 10 digit classes).

<img src="images/neuralnet.png" width="40%"/>

In [ ]:
def load_weights(filedata):
    """ Load the content from data_nn.npy """
    data = np.load(filedata).item()
    theta1 = data['theta1']
    theta2 = data['theta2']
    return theta1, theta2

theta1, theta2 = load_weights('data/weights_nn.npy')
theta1 = np.matrix(theta1)
theta2 = np.matrix(theta2)
print(theta1.shape)
print(theta2.shape)

## Feedforward Propagation and Prediction

Now you will implement feedforward propagation for the neural network. You should implement the feedforward computation that computes $h_\theta(x^{(i)})$ for every example `i` and returns the associated predictions, where the prediction from the neural network will be the label that has the largest output $(h_\theta(x))_k$.

**Implementation Note**: The matrix `X` contains the examples in rows, then you will need to add the column of 1's to the matrix. The matrices `Theta1` and `Theta2` contain the parameters for each unit in rows. Specifically, the first row of `Theta1` corresponds to the first hidden unit in the second layer. When you compute $z^{(2)} = \theta^{(1)}a^{(1)}$, be sure that you index (and if necessary, transpose) `X` correctly so that you get a (`l`) as a column vector.

Remember that the sigmoid function is defined as:

$$g(z) = \frac{1}{1 + e^{-z}}$$

If your predict function using the loaded set of parameters for `Theta1` and `Theta2` is correct, you should see that the accuracy is about 97.5%.

In [4]:
def sigmoid(z):
    """ Activation function """
    # YOUR CODE HERE
    pass
    # return g

def forward(X, theta1, theta2):
    """ 
    Apply the forward propagation 
    To apply backpropagation, the function should return
    the values of a1, z2, a2, z3 and h
    """
    # YOUR CODE HERE
    pass
    #return a1, z2, a2, z3, h

In [ ]:
# Test the forward function
a1, z2, a2, z3, h = forward(X, theta1, theta2)
h.shape

In [ ]:
y_pred = np.array(np.argmax(h, axis=1)+1)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))

print('Accuracy with loaded weights must be 97.52%')
print('Predicted accuracy = %.2f' % (accuracy * 100))

## Cost Function

Now you will implement the cost function and gradient for the neural network. Recall that the cost function for the neural network is

$$J(\theta) = \frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} [-y_k^{(i)}log((h_\Theta(x^{(i)}))_k-(1 - y_k^{(i)})log(1 - h_\Theta(x^{(i)})_k)]$$

where K = 10 is the total number of possible labels. Note that $h_\theta(x^{(i)})_k = a_k^{(3)}$ is the activation (output value) of the k-th output unit. Also, recall that whereas the original labels (in the variable `y`) were 1, 2, ..., 10, for the purpose of training a neural network, we need to recode the labels as vectors containing only values 0 or 1, so that

$$y = \begin{bmatrix}
1 \\ 
0 \\ 
0 \\ 
\vdots \\ 
0 \\
\end{bmatrix},  \ \ \ \
\begin{bmatrix}
0 \\ 
1 \\ 
0 \\ 
\vdots \\ 
0 \\
\end{bmatrix},  \ \ \ \
\begin{bmatrix}
0 \\ 
0 \\ 
1 \\ 
\vdots \\ 
0 \\
\end{bmatrix},  \ \ \ \ \dots \text{or} \ \ \ \
\begin{bmatrix}
0 \\ 
0 \\ 
0 \\ 
\vdots \\ 
1 \\
\end{bmatrix}$$

For example, if $x^{(i)}$ is an image of the digit 5, then the corresponding $y^{(i)}$ (that you should use with the cost function) should be a 10-dimensional vector with $y_5 = 1$, and the other elements equal to 0. You should implement the feedforward computation that computes $h_\theta(x^{(i)})$ for every example `i` and sum the cost over all examples.

In [ ]:
def cost_function(X, y, theta1, theta2):
    """ Verify the cost using theta1 and theta2 """
    # YOUR CODE HERE
    pass
    # return J

In [ ]:
# Test cost function
J = cost_function(X, y_onehot, theta1, theta2)
print('Your current cost should be: 0.287629')
print('Current cost : %f' % J)

# Backpropagation

Now, you will implement the backpropagation algorithm to compute the gradient for the neural network cost function. Once you have computed the gradient, you will be able to train the neural network by minimizing the cost function $J(\theta)$ using an optimizer such as the `fmin_tnc` from Scipy.

You will first implement the backpropagation algorithm to compute the gradients for the parameters for the neural network.

## Sigmoid gradient

First you have to implement the sigmoid gradient function. The gradient for the sigmoid function can be computed as

$$g'(z) = \frac{d}{dz}g(z) = g(z)(1 - g(z))$$

where

$$sigmoid(z) = g(z) = \frac{1}{1+e^{-z}}$$

When you are done, try testing a few values by calling `sigmoid_gradient(z)`. For large values (both positive and negative) of `z`, the gradient should be close to 0. When z = 0, the gradient should be exactly 0.25. Your code should also work with vectors and matrices. For a matrix, your function should perform the sigmoid gradient function on every element.

In [1]:
def sigmoid_gradient(z):
    """ Computes the gradient of the sigmoid function """
    # YOUR CODE HERE
    pass
    # return dg

In [ ]:
# Test the gradient of sigmoid
print('Computing gradient for [1, -0.5, 0, 0.5, 1]')
print('Correct gradients:    [ 0.19661193  0.23500371  0.25        0.23500371  0.19661193]')

values = np.array([1, -0.5, 0, 0.5, 1])
sgrad = sigmoid_gradient(values)

print('Calculated gradients: %s' % str(sgrad))

## Random initialization

When training neural networks, it is important to randomly initialize the parameters for symmetry breaking. One effective strategy for random initialization is to randomly select values for $\Theta^{(l)}$ uniformly in the range [$-\epsilon_{init}$ , $\epsilon_{init}$]. You should use init = 0.12 (One effective strategy for choosing $\epsilon_{init}$ is to base it on the number of units in the network. A good choice of $\epsilon_{init}$ is $\epsilon_{init} = \frac{\sqrt{6}}{\sqrt{L_{in}+L_{out}}}$, where $L_{in} = s_l$ and $L_{out} = s_{l+1}$ are the number of units in the layers adjacent to $\Theta^{(l)}$.) This range of values ensures that the parameters are kept small and makes the learning more efficient. 

Thus, consider a matrix of weights of 10x11, a random initialization should be performed as

$$Theta = rand(10, 11) \times (2\epsilon) - \epsilon$$

In [ ]:
def initialize_weights(L_in, L_out):
    """ 
    Randomly initialize the weights of a layer with 
    L_in income connections and L_out outgoing connections
    
    Note: W should be set to a matrix of size(L_out, 1+L_in) as
          the column row of W handles the 'bias' terms.
    """
    # YOUR CODE HERE
    pass
    #return theta

In [ ]:
print('Initializing Neural Network Parameters...')
# initial setup :: set constants
NB_HIDDEN = 25
LEARNING_RATE = 1.0
input_layer_size = X.shape[1]
num_labels = y_onehot.shape[1]

initial_Theta1 = initialize_weights(input_layer_size, NB_HIDDEN)
initial_Theta2 = initialize_weights(NB_HIDDEN, num_labels)

thetas = np.concatenate((np.ravel(initial_Theta1), np.ravel(initial_Theta2)))

print('Input layer size: %d' % input_layer_size)
print('Number of hidden units: %d' % NB_HIDDEN)
print('Number of labels: %d' % num_labels)
print('theta1 shape: (%d, %d)' % initial_Theta1.shape)
print('theta2 shape: (%d, %d)' % initial_Theta2.shape)

## Backpropagation

Recall that the intuition behind the backpropagation algorithm is as follows. Given a training example ($x^{(t)}, y^{(t)}$), we will first run a "forward pass" to compute all the activations throughout the network, including the output value of the hypothesis $h_\theta(x)$. Then, for each node `j` in layer `l`, we would like to compute an "error term" $\delta_j^{(l)}$ that measures how much that node was "responsible" for any errors in our output.

For an output node, we can directly measure the difference between the network's activation and the true target value, and use that to define $\delta_j^{(3)}$ (since layer 3 is the output layer). For the hidden units, you will compute $\delta_j^{(l)}$ based on a weighted average of the error terms of the nodes in layer $(l+1)$.

In detail, here is the backpropagation algorithm (also depicted in the image below). You should implement steps 1 to 4 in a loop that processes one example at a time. Concretely, you should implement a for-loop `for t in range(1:m)` and place steps 1-4 below inside the for-loop, with the t-th iteration performing the calculation on the t-th training example ($x^{(t)}, y^{(t)}$). Step 5 will divide the accumulated gradients by $m$ to obtain the gradients for the neural network cost function.

<img src="images/backprop.png", width="40%"/>

1) Set the input layer’s values ($a^{(1)}$) to the t-th training example $x^{(t)}$. Perform a feedforward pass (Figure above), computing the activations ($z^{(2)}, a^{(2)}, z^{(3)}, a^{(3)}$) for layers 2 and 3. Note that you need to add a $+1$ term to ensure that the vectors of activations for layers $a^{(1)}$ and $a^{(2)}$ also include the bias unit.

2) For each output unit $k$ in layer 3 (the output layer), set

$$\delta_k^{(3)} = (a_k^{(3)} - y_k)$$

where $y_k \in \{0, 1\}$ indicates whether the current training example belongs to class $k (y_k = 1)$, or if it belongs to a different class $(y_k = 0)$. You may find logical arrays helpful for this task (explained in the previous programming exercise).

3) For the hidden layer $l=2$, set

$$\delta^{(2)} = (\Theta^{(2)})^T\delta^{(3)} .* g'(z^{(2)})$$

4) Accumulate the gradient from this example using the following formula. Note that you should skip or remove $\delta_0^{(2)}$ .

$$\Delta^{(l)} = \Delta^{(l)} + \delta^{(l+1)}(a^{(1)})^T$$

5) Obtain the gradient for the neural network cost function by dividing the accumulated gradients by $\frac{1}{m}$:

$$\frac{\partial}{\partial\Theta_{ij}^{(l)}}J(\Theta) = D_{ij}^{(l)} = \frac{1}{m}\Delta_{ij}^{(l)}$$

**Tip**: You should implement the backpropagation algorithm only after you have successfully completed the feedforward and cost functions. While implementing the backpropagation algorithm, it is often useful to use the size function to print out the sizes of the variables you are working with if you run into dimension mismatch errors.

In [ ]:
# Backpropagation function
def backpropagation(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    """ 
    Apply backpropagation algorithm on the input. Backpropagation has this signature
    due to `scipy.optimize.minimize` function that will perform it automatically.
    """
    # YOUR CODE HERE
    pass

    #grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    #return J, grad

In [ ]:
# Perform forward-backpropagation, updating thetas to minimize the cost function
from scipy.optimize import minimize

# Minimize a scalar function of one or more variables using a truncated Newton (TNC) algorithm
# If jac is a Boolean and is True, fun is assumed to return the gradient along with the objective function
fmin = minimize(fun=backpropagation, x0=thetas, 
                args=(input_layer_size, NB_HIDDEN, num_labels, X, y_onehot, LEARNING_RATE), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

In [ ]:
# Compute the new accuracy for the trained model
y_pred = np.array(np.argmax(h, axis=1)+1)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))

print('Accuracy with loaded weights must be 97.52%')
print('Predicted accuracy = %.2f' % (accuracy * 100))